In [3]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    #Closing() closes the thing (the url in this case) upon completion of the block
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    #the below sets it so that the content_type (ie. json, html, etc.) is not none. .Find finds the index position
    #of html, if there is one, and if its greater than -1 (it has an index position) it returns true
    #I beleive 200 is a good status code
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [4]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

The simple_get() function accepts a single url argument. It then makes a GET request to that URL. If nothing goes wrong, you end up with the raw HTML content for the page you requested. If there were any problems with your request (like the URL is bad, or the remote server is down), then your function returns None.

The closing() function ensures that any network resources are freed when they go out of scope in that with block. Using closing() like that is good practice and helps to prevent fatal errors and network timeouts.

In [10]:
#from mathematicians import simple_get
raw_html = simple_get('https://realpython.com/blog/')
len(raw_html)

473472

In [11]:
no_html = simple_get('https://realpython.com/blog/nope-not-gonna-find-it')

Wrangling HTML With BeautifulSoup

Once you have raw HTML in front of you, you can start to select and extract. For this purpose, you will be using BeautifulSoup. The BeautifulSoup constructor parses raw HTML strings and produces an object that mirrors the HTML document’s structure. The object includes a slew of methods to select, view, and manipulate DOM nodes and text content.

In [14]:
#this is a quick example of an html file I saved
raw_html = open('contrived.html').read()
html = BeautifulSoup(raw_html, 'html.parser')
for p in html.select('p'):
    if p['id'] == 'walrus':
        print(p.text)

 I am the walrus 


Now that you have given the select() method in BeautifulSoup a short test drive, how do you find out what to supply to select()? The fastest way is to step out of Python and into your web browser’s developer tools. You can use your browser to examine the document in some detail. I usually look for id or class element attributes or any other information that uniquely identifies the information I want to extract.

To make matters concrete, turn to the list of mathematicians you saw earlier. If you spend a minute or two looking at this page’s source, you can see that each mathematician’s name appears inside the text content of an >li< tag. To make matters even simpler, >li< tags on this page seem to contain nothing but names of mathematicians.

In [12]:
#so here we are getting the mathmeticians names - if you use CTRL + U on the webpage you can see the source data
raw_html = simple_get('http://www.fabpedigree.com/james/mathmen.htm')
html = BeautifulSoup(raw_html, 'html.parser')
for i, li in enumerate(html.select('li')):
        print(i, li.text)

0  Isaac Newton
 Archimedes
 Carl F. Gauss
 Leonhard Euler
 Bernhard Riemann

1  Archimedes
 Carl F. Gauss
 Leonhard Euler
 Bernhard Riemann

2  Carl F. Gauss
 Leonhard Euler
 Bernhard Riemann

3  Leonhard Euler
 Bernhard Riemann

4  Bernhard Riemann

5  Henri Poincaré
 Joseph-Louis Lagrange
 Euclid  of Alexandria
 David Hilbert
 Gottfried W. Leibniz

6  Joseph-Louis Lagrange
 Euclid  of Alexandria
 David Hilbert
 Gottfried W. Leibniz

7  Euclid  of Alexandria
 David Hilbert
 Gottfried W. Leibniz

8  David Hilbert
 Gottfried W. Leibniz

9  Gottfried W. Leibniz

10  Alexandre Grothendieck
 Pierre de Fermat
 Évariste Galois
 John von Neumann
 René Descartes

11  Pierre de Fermat
 Évariste Galois
 John von Neumann
 René Descartes

12  Évariste Galois
 John von Neumann
 René Descartes

13  John von Neumann
 René Descartes

14  René Descartes

15  Karl W. T. Weierstrass
 Srinivasa Ramanujan
 Hermann K. H. Weyl
 Peter G. L. Dirichlet
 Niels Abel

16  Srinivasa Ramanujan
 Hermann K. H. Weyl
 

The above experiment shows that some of the >li< elements contain multiple names separated by newline characters, while others contain just a single name. With this information in mind, you can write your function to extract a single list of names:

In [16]:
def get_names():
    """
    Downloads the page where the list of mathematicians is found
    and returns a list of strings, one per mathematician
    """
    url = 'http://www.fabpedigree.com/james/mathmen.htm'
    response = simple_get(url)

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        names = set()
        for li in html.select('li'):
            for name in li.text.split('\n'):
                if len(name) > 0:
                    names.add(name.strip())
        return list(names)

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))


The get_names() function downloads the page and iterates over the >li< elements, picking out each name that occurs. Next, you add each name to a Python set, which ensures that you don’t end up with duplicate names. Finally, you convert the set to a list and return it

Again, you should first check out an example page in your browser’s developer tools. It looks as if the text appears inside an >a< element, and the href attribute of that element always contains the string 'latest-60' as a substring. That’s all the information you need to write your function:

In [17]:
def get_hits_on_name(name):
    """
    Accepts a `name` of a mathematician and returns the number
    of hits that mathematician's Wikipedia page received in the 
    last 60 days, as an `int`
    """
    # url_root is a template string that is used to build a URL.
    url_root = 'URL_REMOVED_SEE_NOTICE_AT_START_OF_ARTICLE'
    response = simple_get(url_root.format(name))

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')

        hit_link = [a for a in html.select('a')
                    if a['href'].find('latest-60') > -1]

        if len(hit_link) > 0:
            # Strip commas
            link_text = hit_link[0].text.replace(',', '')
            try:
                # Convert to integer
                return int(link_text)
            except:
                log_error("couldn't parse {} as an `int`".format(link_text))

    log_error('No pageviews found for {}'.format(name))
    return None

In [18]:
if __name__ == '__main__':
    print('Getting the list of names....')
    names = get_names()
    print('... done.\n')

    results = []

    print('Getting stats for each name....')

    for name in names:
        try:
            hits = get_hits_on_name(name)
            if hits is None:
                hits = -1
            results.append((hits, name))
        except:
            results.append((-1, name))
            log_error('error encountered while processing '
                      '{}, skipping'.format(name))

    print('... done.\n')

    results.sort()
    results.reverse()

    if len(results) > 5:
        top_marks = results[:5]
    else:
        top_marks = results

    print('\nThe most popular mathematicians are:\n')
    for (mark, mathematician) in top_marks:
        print('{} with {} pageviews'.format(mathematician, mark))

    no_results = len([res for res in results if res[0] == -1])
    print('\nBut we did not find results for '
          '{} mathematicians on the list'.format(no_results))

Getting the list of names....
... done.

Getting stats for each name....
Error during requests to URL_REMOVED_SEE_NOTICE_AT_START_OF_ARTICLE : Invalid URL 'URL_REMOVED_SEE_NOTICE_AT_START_OF_ARTICLE': No schema supplied. Perhaps you meant http://URL_REMOVED_SEE_NOTICE_AT_START_OF_ARTICLE?
No pageviews found for Adrien M. Legendre
Error during requests to URL_REMOVED_SEE_NOTICE_AT_START_OF_ARTICLE : Invalid URL 'URL_REMOVED_SEE_NOTICE_AT_START_OF_ARTICLE': No schema supplied. Perhaps you meant http://URL_REMOVED_SEE_NOTICE_AT_START_OF_ARTICLE?
No pageviews found for George D. Birkhoff
Error during requests to URL_REMOVED_SEE_NOTICE_AT_START_OF_ARTICLE : Invalid URL 'URL_REMOVED_SEE_NOTICE_AT_START_OF_ARTICLE': No schema supplied. Perhaps you meant http://URL_REMOVED_SEE_NOTICE_AT_START_OF_ARTICLE?
No pageviews found for Archytas  of Tarentum
Error during requests to URL_REMOVED_SEE_NOTICE_AT_START_OF_ARTICLE : Invalid URL 'URL_REMOVED_SEE_NOTICE_AT_START_OF_ARTICLE': No schema supplied.

In [20]:
raw_html = simple_get('https://www.allrecipes.com/recipe/82347/best-ever-crab-cakes/?internalSource=popular&referringContentType=Homepage')

In [87]:
html = BeautifulSoup(raw_html, 'html.parser')
name = set()
for i, label in enumerate(html.select('label')):
    name.add(label.get('title'))
print(name)

{'1 tablespoon butter', '1 egg', None, '1/8 teaspoon red pepper flakes', '8 ounces crabmeat', '1/2 cup crushed buttery round crackers', '4 teaspoons lemon juice', '1 teaspoon dried tarragon', '3 tablespoons mayonnaise', '1 tablespoon minced green onions'}


In [114]:
def get_ingredients():
    """
    Downloads the page where the list of mathematicians is found
    and returns a list of strings, one per mathematician
    """
    response = raw_html

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        names = set()
        for label in html.select('label'):
            if len(name) > 0:
                names.add(label.get('title'))
        return list(names)

def get_names():
    """
    Downloads the page where the list of mathematicians is found
    and returns a list of strings, one per mathematician
    """
    url = raw_html
    response = simple_get(url)

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        names = set()
        for label in html.select('label'):
                if len(name) > 0:
                    names.add(name.strip())
        return list(names)

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))


In [115]:
ingredients = get_ingredients()
print(ingredients)

['1 tablespoon butter', '1 egg', None, '1/8 teaspoon red pepper flakes', '8 ounces crabmeat', '1/2 cup crushed buttery round crackers', '4 teaspoons lemon juice', '1 teaspoon dried tarragon', '3 tablespoons mayonnaise', '1 tablespoon minced green onions']


In [44]:
if __name__ == '__main__':
    print('Getting the list of ingredients....')
    names = get_ingredients()
    print('... done.\n')

    results = []

    print('Getting stats for each name....')
"""
    for name in names:
        try:
            hits = get_hits_on_name(name)
            if hits is None:
                hits = -1
            results.append((hits, name))
        except:
            results.append((-1, name))
            log_error('error encountered while processing '
                      '{}, skipping'.format(name))

    print('... done.\n')

    results.sort()
    results.reverse()

    if len(results) > 5:
        top_marks = results[:5]
    else:
        top_marks = results

    print('\nThe most popular mathematicians are:\n')
    for (mark, mathematician) in top_marks:
        print('{} with {} pageviews'.format(mathematician, mark))

    no_results = len([res for res in results if res[0] == -1])
    print('\nBut we did not find results for '
          '{} mathematicians on the list'.format(no_results))
"""

Getting the list of ingredients....
Error during requests to b'\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="en-us">\r\n<head>\r\n    <title>Best Ever Crab Cakes Recipe - Allrecipes.com</title>\r\n\r\n<script src=\'https://secureimages.allrecipes.com/assets/deployables/v-1.169.0.5113/karma.bundled.js\' async=true></script>\r\n\r\n\r\n    <!--Make our website baseUrl available to the client-side code-->\r\n    <script type="text/javascript">\r\n        var AR = AR || {};\r\n\r\n        AR.segmentWriteKey = "RnmsxUrjIjM7W62olfjKgJrcsVlxe68V";\r\n        AR.baseWebsiteUrl = \'https://www.allrecipes.com\';\r\nwindow.dataLayer={"version":"1.0","pageInstanceId":"www.allrecipes.com/recipe/82347/best-ever-crab-cakes/","externalLinkId":"","page":{"pageInfo":{"pageId":"82347","pageName":"/recipe/82347/best-ever-crab-cakes/","destinationUrl":"https://www.allrecipes.com/recipe/82347/best-ever-crab-cakes/?internalSource=popular\\u0026referringContentType=Homepage","sysEnv":"RD0003FFA86B09","variant":"

Exception: Error retrieving contents at b'\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="en-us">\r\n<head>\r\n    <title>Best Ever Crab Cakes Recipe - Allrecipes.com</title>\r\n\r\n<script src=\'https://secureimages.allrecipes.com/assets/deployables/v-1.169.0.5113/karma.bundled.js\' async=true></script>\r\n\r\n\r\n    <!--Make our website baseUrl available to the client-side code-->\r\n    <script type="text/javascript">\r\n        var AR = AR || {};\r\n\r\n        AR.segmentWriteKey = "RnmsxUrjIjM7W62olfjKgJrcsVlxe68V";\r\n        AR.baseWebsiteUrl = \'https://www.allrecipes.com\';\r\nwindow.dataLayer={"version":"1.0","pageInstanceId":"www.allrecipes.com/recipe/82347/best-ever-crab-cakes/","externalLinkId":"","page":{"pageInfo":{"pageId":"82347","pageName":"/recipe/82347/best-ever-crab-cakes/","destinationUrl":"https://www.allrecipes.com/recipe/82347/best-ever-crab-cakes/?internalSource=popular\\u0026referringContentType=Homepage","sysEnv":"RD0003FFA86B09","variant":"Control","abTestName"

In [74]:
url = raw_html
response = simple_get(url)

if response is not None:
    html = BeautifulSoup(response, 'html.parser')
    names = []
    for i, label in enumerate(html.select('label')):
        names.add(label)
    print(names)

Error during requests to b'\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="en-us">\r\n<head>\r\n    <title>Best Ever Crab Cakes Recipe - Allrecipes.com</title>\r\n\r\n<script src=\'https://secureimages.allrecipes.com/assets/deployables/v-1.169.0.5113/karma.bundled.js\' async=true></script>\r\n\r\n\r\n    <!--Make our website baseUrl available to the client-side code-->\r\n    <script type="text/javascript">\r\n        var AR = AR || {};\r\n\r\n        AR.segmentWriteKey = "RnmsxUrjIjM7W62olfjKgJrcsVlxe68V";\r\n        AR.baseWebsiteUrl = \'https://www.allrecipes.com\';\r\nwindow.dataLayer={"version":"1.0","pageInstanceId":"www.allrecipes.com/recipe/82347/best-ever-crab-cakes/","externalLinkId":"","page":{"pageInfo":{"pageId":"82347","pageName":"/recipe/82347/best-ever-crab-cakes/","destinationUrl":"https://www.allrecipes.com/recipe/82347/best-ever-crab-cakes/?internalSource=popular\\u0026referringContentType=Homepage","sysEnv":"RD0003FFA86B09","variant":"Control","abTestName":"RDP_Video_Ste

In [75]:
print(html)


<!DOCTYPE html>

<html lang="en-us">
<head>
<title>Best Ever Crab Cakes Recipe - Allrecipes.com</title>
<script async="true" src="https://secureimages.allrecipes.com/assets/deployables/v-1.169.0.5113/karma.bundled.js"></script>
<!--Make our website baseUrl available to the client-side code-->
<script type="text/javascript">
        var AR = AR || {};

        AR.segmentWriteKey = "RnmsxUrjIjM7W62olfjKgJrcsVlxe68V";
        AR.baseWebsiteUrl = 'https://www.allrecipes.com';
window.dataLayer={"version":"1.0","pageInstanceId":"www.allrecipes.com/recipe/82347/best-ever-crab-cakes/","externalLinkId":"","page":{"pageInfo":{"pageId":"82347","pageName":"/recipe/82347/best-ever-crab-cakes/","destinationUrl":"https://www.allrecipes.com/recipe/82347/best-ever-crab-cakes/?internalSource=popular\u0026referringContentType=Homepage","sysEnv":"RD0003FFA86B09","variant":"Control","abTestName":"RDP_Video_Step","version":"20190709","issueDate":"07/23/2019 16:43:05","effectiveDate":"07/23/2019 16:43:05","

In [81]:
html.label

KeyError: 'Meal Type'